In [31]:
import pandas as pd
import numpy as np
import os
import csv

cwd = os.getcwd()


In [32]:
#!cp /Users/saulgrimaldo1/Downloads/create_training_data.py code/.
!cat dta/training_data_combined.txt | python code/create_training_data.py > dta/training_data_new.txt

In [34]:
!tail -n 100 dta/training_data_new.txt

984161864094552069|@BridgeeCee @Robski80 @lfstn @SportingNest @Suttynotsweep @Jakesbarleeds Yeah I can hear your   🙄 , it’s so true tho didn’t have all these problems with the older generations , people just got on with there lives !|BridgeeCee|930460831|984161304373088257|1
984161900115218432|2 weeks left of this semester and I’m REALLY questioning if I’ll make it out alive. Not  . My survival-Its touch &amp; go.|None|None|None|1
984161926413520896|Did Darlene &amp; Becky switch characters on #Roseanne? Wasn’t Becky the smart, sensible one and Darlene the   one? I thought she would be a lawyer not a waitress. But I must admit I am enjoying @RoseanneOnABC tremendously.|None|None|None|1
984161938539208705|@CiscuLog 💙💙💙 (I hope thats not   bc if it is were gonna have trouble)|CiscuLog|3383568557|984161420467228672|1
984161954133659649|@mheadlam Im not.    is hard to convey on Twitter.|mheadlam|41352789|984155266185940992|1
984162045858852871|@RlCHJIHAN lmaoao thank u sorry iM just   but 

In [19]:
!head dta/training_data_new.txt

id|text|in_reply_to_screen_name|in_reply_to_user_id|in_reply_to_status_id|sarcastic
977821226243706880|RT @TomHall: Nothing like a Hot Bathto kickoff a great day!💦 🛀🏽 #SundayMorning #Otter #Nature #Work #SundayFunday https://t.co/W6mP…|None|None|None|0
977812056631140352|RT @bethereumteam: #Bethereum makes any sports event more fun! ⚽Watch the video of the betting process and tag the friends youd like to c…|None|None|None|0
977817505069715456|Is it time to embark on your next cruise? Not sure what to choose? Come along to @bewlwater on Sunday 29April for the #CruiseInspirations Show. FREE ENTRY. #WIN a CRUISE https://t.co/gDUt5eEZqv https://t.co/PX6XBtfg3U|None|None|None|0
977811710299070465|#crypto #bitcoin #followback #ico #folowme #bounty #blockchain #eth $MBRS $DOM $IDEA $TOV $SLIP $KNOW $NIO https://t.co/ijepjEqRJO|None|None|None|0
977810518596636673|RT @agedchild003: RT @dkusliy: RT @dawidczagan: Hacking iOS apps: static code analysis and recon https://t.co/LHw8cul1zo #mobileSecu

In [35]:
#df_sarcastic = pd.read_csv('merged_data_v1_1.csv', sep = '|')

df = pd.DataFrame()



with open('dta/training_data_new.txt', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            header= row
            df = pd.DataFrame(columns = row)
        else:
            if len(header) != len(row):
                continue
            else:
                addition = {i:j for i, j in zip(header,row)}
                df = df.append(addition, ignore_index=True )
                


df.tail()        

,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic
13493,984164458820964359,RT @johnniejae: which then gives me time to ca...,None,None,None,1
13494,984164486859907073,@Tracker_TD Obvious is obvious,Chris_RWS,1453396112,984164139584114690,1
13495,984164502814928896,@ummmno21 Yeah it appears that inside jokes an...,ummmno21,798603640613335040,984149821291458561,1
13496,984164534620475392,"@meknt_ Yeah, youre CLEARLY talking to all the...",meknt_,4708806988,984163505854173185,1
13497,984164553528311810,"❝ / is / a thing, Yknow. Since you use so muc...",None,None,None,1


In [36]:
for i in np.unique(df['in_reply_to_status_id']):
    if len(i) != 18:
        print (i)

8522329478
None


In [37]:
#### true exmaples : 
### false examples :
import re
status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_storage = []

for j in df[df['in_reply_to_status_id'] != 'None'].index:
    if (((df['in_reply_to_screen_name'][j][0] != '#')) & 
        bool(re.match(status_id_pattern,df['in_reply_to_status_id'][j]))):
        index_storage.append(j)
    else: continue

len(index_storage)

3322

In [38]:
#want the clean rows that do not have retreats

status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_alt = []

for j in df[df['in_reply_to_status_id'] == 'None'].index:
    if (bool(re.match(status_id_pattern,df['id'][j]))):
        index_alt.append(j)
    else: continue

len(index_alt)


10175

In [39]:
#subset those data sets that will not have as much trouble

sub_df = df.iloc[index_storage]
sub_df['url']= "https://twitter.com/"+sub_df['in_reply_to_screen_name']+"/status/"+sub_df['in_reply_to_status_id']
#max(index_storage)
sub_df.head()

/home/saulgrimaldo1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url
96,977780264612061184,@derekmagill @rogerkver @Satoshis_Vision Sound...,derekmagill,1608992342,977405844349333504,0,https://twitter.com/derekmagill/status/9774058...
112,977784274806976513,@eclecticbrotha https://t.co/fxlFDEum2wGraphic...,eclecticbrotha,300642262,977701797799264257,0,https://twitter.com/eclecticbrotha/status/9777...
161,977780600831528960,@T_Roo_Mur https://t.co/9amI3T8qV4Graphic Desi...,T_Roo_Mur,315202249,977528739625078785,0,https://twitter.com/T_Roo_Mur/status/977528739...
227,977780026266529793,@MollyJongFast @marcorubio When you took the #...,MollyJongFast,14298769,977745221650919424,0,https://twitter.com/MollyJongFast/status/97774...
257,977807086770839552,@PerthGloryFC @danielstynes11 #are you #lookin...,PerthGloryFC,269648315,977806055005499393,0,https://twitter.com/PerthGloryFC/status/977806...


In [40]:
sub_df

,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url
96,977780264612061184,@derekmagill @rogerkver @Satoshis_Vision Sound...,derekmagill,1608992342,977405844349333504,0,https://twitter.com/derekmagill/status/9774058...
112,977784274806976513,@eclecticbrotha https://t.co/fxlFDEum2wGraphic...,eclecticbrotha,300642262,977701797799264257,0,https://twitter.com/eclecticbrotha/status/9777...
161,977780600831528960,@T_Roo_Mur https://t.co/9amI3T8qV4Graphic Desi...,T_Roo_Mur,315202249,977528739625078785,0,https://twitter.com/T_Roo_Mur/status/977528739...
227,977780026266529793,@MollyJongFast @marcorubio When you took the #...,MollyJongFast,14298769,977745221650919424,0,https://twitter.com/MollyJongFast/status/97774...
257,977807086770839552,@PerthGloryFC @danielstynes11 #are you #lookin...,PerthGloryFC,269648315,977806055005499393,0,https://twitter.com/PerthGloryFC/status/977806...
313,977808109392494592,@etribune warm wishes to both teams of #PSL201...,etribune,125944392,977807172787728384,0,https://twitter.com/etribune/status/9778071727...
327,977817677338152960,@theginguide @pinckneysgin COMPETITION! Win a ...,theginguide,2959246755,977528719525994496,0,https://twitter.com/theginguide/status/9775287...
605,977784005159522304,@NRA The idea is to shift public opinion to su...,NRA,21829541,977197022938836992,0,https://twitter.com/NRA/status/977197022938836992
623,977808413202812933,@TemiOyewole Were rehabilitating terroristsPay...,JudeFeranmi,602752755,977807742768467968,0,https://twitter.com/JudeFeranmi/status/9778077...
678,977784370340777984,@Pam2189 @GIJoeOPS The fact that she chose to ...,Pam2189,775082681608482816,977707461082726400,0,https://twitter.com/Pam2189/status/97770746108...


In [41]:
from bs4 import BeautifulSoup
import requests
import re
import json
def preceding_tweet(URL_):

    ### TODO  URL_ will be a function of the table 
    #URL_ = "https://twitter.com/verobchfinfan/status/976155584448868353"
    page = requests.get(URL_)

    soup = BeautifulSoup(page.content, 'html.parser')

    html = list(soup.children)[2]

    body = list(html.children)[3]


    ###  entity in tweet that gives a full tweet with low confidence in structure

    p1 = list(body.children)[61]

    p1a = list(p1.children)[5]
    p1b=list(list(p1a.children)[3].children)[1]
    set_of_parent = set(list(p1b.children)[1].get_text().split('\n'))
    set_of_parent = list(map(lambda x: x.strip(),set_of_parent))

   
    ###  entity in tweet that gives a partial tweet with high confidence in structure
    p2 = list(body.children)[65]

    p2a= json.loads(p2.__dict__['attrs']['value'])

    p2a['initialState']['title']

    pattern_1 = r'^[A-Za-z0-9 ]* on Twitter: "'
    pattern_2 = '\u2026'

    if re.match(pattern_1, p2a['initialState']['title']):
        start_index = len(re.match(pattern_1, p2a['initialState']['title']).group(0))
    else:
        start_index = None
    
    
    if pattern_2 in p2a['initialState']['title']:
        stop_index = p2a['initialState']['title'].rfind(pattern_2)
    else: 
        stop_index = None

    partial_tweet = p2a['initialState']['title'][start_index:stop_index]
    
    
    #### pull out tweet preceding

    potential_tweet_list = [el for el in set_of_parent if partial_tweet in el]

    len_counter = 0

    for i in potential_tweet_list:
        current_len = len(i)
        if current_len> len_counter:
            preceding_tweet_x = i
        else:
            pass
    if len(potential_tweet_list)==0:
        return partial_tweet
    else:
        return preceding_tweet_x

In [42]:
storage_pretweet = []

for x in sub_df['url']:
    url = x.strip()
    try:
        storage_pretweet.append(preceding_tweet(url))
    except:
        storage_pretweet.append(None)
    

In [43]:
storage_pretweet

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [44]:
sub_df['first_preceding_tweet'] = storage_pretweet

sub_df.head()

/home/saulgrimaldo1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url,first_preceding_tweet
96,977780264612061184,@derekmagill @rogerkver @Satoshis_Vision Sound...,derekmagill,1608992342,977405844349333504,0,https://twitter.com/derekmagill/status/9774058...,None
112,977784274806976513,@eclecticbrotha https://t.co/fxlFDEum2wGraphic...,eclecticbrotha,300642262,977701797799264257,0,https://twitter.com/eclecticbrotha/status/9777...,None
161,977780600831528960,@T_Roo_Mur https://t.co/9amI3T8qV4Graphic Desi...,T_Roo_Mur,315202249,977528739625078785,0,https://twitter.com/T_Roo_Mur/status/977528739...,None
227,977780026266529793,@MollyJongFast @marcorubio When you took the #...,MollyJongFast,14298769,977745221650919424,0,https://twitter.com/MollyJongFast/status/97774...,None
257,977807086770839552,@PerthGloryFC @danielstynes11 #are you #lookin...,PerthGloryFC,269648315,977806055005499393,0,https://twitter.com/PerthGloryFC/status/977806...,None


In [45]:
index_alt

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,

In [46]:
merged_data = pd.concat([df.iloc[index_alt],sub_df])

print(merged_data.shape)

merged_data.tail(10)

(13497, 8)


,first_preceding_tweet,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,sarcastic,text,url
13480,None,984164011959824389,DouglasLNorton,981178102960349184,1058734063,1,@DouglasLNorton @RealMAGASteve @ricktator417 @...,https://twitter.com/DouglasLNorton/status/9811...
13483,None,984164208169320448,MultaniSaint,984163578214154240,296665817,1,@MultaniSaint No I told her its a terrible pla...,https://twitter.com/MultaniSaint/status/984163...
13485,None,984164342563209216,ifenn,984162400306958338,60163,1,"Rather than the usual , ridicule, shame, and ...",https://twitter.com/ifenn/status/9841624003069...
13486,None,984164352902213632,schristakos,984073637224439808,348862752,1,@schristakos @TenticalsIn @crypt_ghost @hortau...,https://twitter.com/schristakos/status/9840736...
13488,None,984164357587124225,Ayanfx09,984159473680891904,129719417,1,@Ayanfx09 @hshaban The internet desperately ne...,https://twitter.com/Ayanfx09/status/9841594736...
13490,None,984164365535514625,asteven79,984163756560203776,366371143,1,@asteven79 @JinxTheSavage @Galeon_98 @KOSDFF Y...,https://twitter.com/asteven79/status/984163756...
13491,None,984164396757745664,nicholasfilipas,984164176820977664,786050935,1,@nicholasfilipas Of course it is. Look at my ...,https://twitter.com/nicholasfilipas/status/984...
13494,None,984164486859907073,Chris_RWS,984164139584114690,1453396112,1,@Tracker_TD Obvious is obvious,https://twitter.com/Chris_RWS/status/984164139...
13495,None,984164502814928896,ummmno21,984149821291458561,798603640613335040,1,@ummmno21 Yeah it appears that inside jokes an...,https://twitter.com/ummmno21/status/9841498212...
13496,None,984164534620475392,meknt_,984163505854173185,4708806988,1,"@meknt_ Yeah, youre CLEARLY talking to all the...",https://twitter.com/meknt_/status/984163505854...


In [51]:
import re
def clean_text(text):
    text = re.sub(r"(\||\'|\"|\n|#sarcasm|#sarcastic)"," ", str(text),flags=re.IGNORECASE)
    return text

In [55]:
merged_data['first_preceding_tweet_clean'] = merged_data['first_preceding_tweet'].apply(clean_text)


merged_data_clean = merged_data[['text', 'first_preceding_tweet_clean','sarcastic']]

merged_data_clean.to_csv("dta/merged_data_v4.csv", sep = '|', index = False)

In [56]:
merged_data_clean.head(10)

,text,first_preceding_tweet_clean,sarcastic
0,RT @Mighty_1ne: GR8NESS • Come CloserGo check ...,nan,0
1,#blockchain #btc #crypto #cryptocurrency #bitc...,nan,0
2,RT @7David_Seven: The fear of every politician...,nan,0
3,Get Carton for FREE on https://t.co/5OZjqMHyBg...,nan,0
4,RT @makmu1313: One of my pics from today! I’m ...,nan,0
5,#Sports #scores WATCH: Wilds Zucker scores 30...,nan,0
6,RT @guerrero_bu: #Twitter off Me voy al desm...,nan,0
7,New Unknown Species discovered in Africa https...,nan,0
8,RT @IOL: #Education: Why we should keep corrup...,nan,0
9,RT @Actor_Siddharth: #Kammarasambhavam #Histor...,nan,0


In [57]:
!wc -l dta/merged_data_v4.csv

13498 dta/merged_data_v4.csv
